#**Get Info For Dataset**

There are different types of datasets which we can use to fine-tune Large Language Models.

<a href="https://colab.research.google.com/github/kyledinh/slowblood/blob/main/notebookds/nb_get_info_for_dataset.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

- `https://raw.githubusercontent.com/kyledinh/slowblood/main/notebooks/nb_get_info_for_dataset.ipynb`

#**00. VARIABLES FOR THIS NOTEBOOK**

In [ ]:
## LOAD THE VARIABLES HERE FOR THE base_model, dataset, tokenizer 

BASE_DATASET = "garage-bAInd/Open-Platypus"
NEW_DATASET = "Slowblood/mini-platypus-two"

BASE_MODEL = "NousResearch/Llama-2-7b-hf"
MODEL_TOKENIZER = "NousResearch/Llama-2-7b-hf"
EMBEDDING_MODEL = "thenlper/gte-large"

DS_HEADER_INSTRUCTION = "instruction"
DS_HEADER_INPUT = "input"
DS_HEADER_OUTPUT = "output"

#**01. Install All the Required Libraries**

In [ ]:
#Install Datasets library to load the dataset from hugging face into the Google Colab Notebook.
#Install Transformers library to import the Autotokenizer this will convert the raw text into tokens
#Install Sentence Transformers Library to download the Embedding Model
%pip install -q datasets transformers sentence_transformers faiss-gpu

#**02. Login to Hugging Face with Token**

In [ ]:
# Required when training models/data that are gated on HuggingFace, and required for pushing models to HuggingFace
%pip install huggingface_hub
from huggingface_hub import notebook_login

notebook_login()

#**04. Load the Dataset**

In [ ]:
from datasets import load_dataset
dataset = load_dataset(BASE_DATASET) ## Defined in 03. Set Variable 

# Read the Dataset as the Pandas Dataset
dataset["train"].to_pandas()

#**05. Analyze the Dataset**

In [ ]:
%pip install matplotlib seaborn

#Use Transformers library to import the Autotokenizer this will convert the raw text into tokens
from transformers import AutoTokenizer
# Import Matplotlib and Seaborn Library to plot and visualize the data
import matplotlib.pyplot as plt
import seaborn as sns
#Next I will load the Tokenizer from Llama 2, I will use the Nous Research Version of Llama 2 and not the official one from meta,
# to use the official version of Llama 2 from meta you need to have an Hugging Face pro account
tokenizer = AutoTokenizer.from_pretrained(MODEL_TOKENIZER) ## Defined in 03. Set Variable

In [ ]:
#Tokenizer Downloaded
#Tokenize each row in the Instruction and Output Columns in the  Dataset and Count the Total Number of Tokens
instruction_tokens_count = [len(tokenizer.tokenize(example[DS_HEADER_INSTRUCTION])) for example in dataset["train"]]
print("Instruction Tokens Count", instruction_tokens_count)
print("Length of Instruction Tokens Count", len(instruction_tokens_count))

output_tokens_count = [len(tokenizer.tokenize(example[DS_HEADER_OUTPUT])) for example in dataset["train"]]
print("Output Tokens Count", output_tokens_count)
print("Length of the Output Tokens Count", len(output_tokens_count))

#Combine the Instruction Token Count and the Output Token Count
combine_tokens_count = [instruction + output for instruction, output in zip(instruction_tokens_count, output_tokens_count)]
print("Combine Tokens Count", combine_tokens_count)
print("Length of the Combine Tokens Count", len(combine_tokens_count))

In [ ]:
# Create a Histogram using the Matplotlib Library to see the distribution of our token counts
def plot_distribution(tokens_count, title):
  sns.set_style("whitegrid")
  plt.figure(figsize=(15, 6))
  plt.hist(tokens_count, bins=50, color='#3498db', edgecolor='black')
  plt.title(title, fontsize=16)
  plt.xlabel("Number of Tokens", fontsize=14)
  plt.ylabel("Number of Examples", fontsize=14)
  plt.xticks(fontsize=12)
  plt.yticks(fontsize=12)
  plt.tight_layout()
  plt.show()

#Insruction Tokens Count
plot_distribution(instruction_tokens_count, "Distribution of the tokens count for Instruction Column")
#Output Tokens Count
plot_distribution(output_tokens_count, "Distribution of the tokens count for Output Column")
#Combine Tokens Count
plot_distribution(combine_tokens_count, "Distribution of the tokens count for combined Instruction + Output Column")
#The mean is around 500 tokens but there is a long tail distribution which goes up to 5000 tokens

**Now we have find out the Number of Tokens in the Instruction Column and in the Output Column and Combine Instruction and Output Column**
But the Question remains is why we need to know the number of tokens because the Llama 2 and other LLMs, have a certain context window (input tokens limit)(Maximum Context Size of Llama 2 by default is 4096)  and if the tokens goes beyond the this context window then it is not going to be helpful. So, its important to know the number of tokens in our dataset

#**06. Filter out rows with more than 2048 tokens in the Combine Token Count (Instruction Column + Output Column)**

**Maximum Context Size of Llama 2 by default = 4096 tokens**

In [ ]:
#We will remove samples with more than 2048 tokens in the Combine Token Count
#First, filter  rows with less or equal to 2048 tokens
valid_indices = [i for i, count in enumerate(combine_tokens_count) if count <= 2048]
print(f"Number of Valid Rows: {len(valid_indices)}")
#Number of Rows with more than 2048 tokens
print(f"Removing: {len(dataset['train']) - len(valid_indices)} rows....")
#Second, extract valid rows based on indices
dataset['train'] = dataset['train'].select(valid_indices)

#Get token counts for valid rows
token_counts = [combine_tokens_count[i] for i in valid_indices]

plot_distribution(token_counts, "New distribution of Combine Total Count (Instruction + Output Columns)")

In [ ]:
dataset["train"].to_pandas()

**The original dataset has 24926 rows × 4 columns now the number of rows have been decreased  to 24895 rows × 4 columns**

#**07. Near-deduplication using Embeddings**

You can check the [MTEB leaderboard](https://huggingface.co/spaces/mteb/leaderboard) on Hugging face to choose the embedding model

In [ ]:
# In this notebook i am using the gte-large embedding model its not the best embedding model but it is fast
#I will use the sentence transformers library to download the embedding model
from sentence_transformers import SentenceTransformer
#Faiss will be used as our vector database which is not very fast but very simple to use
import faiss
from datasets import Dataset, DatasetDict
#tqdm creates a nice loading bar
from tqdm.autonotebook import tqdm
import numpy as np

def deduplicate_dataset(dataset: Dataset, model: str, threshold: float):
  #Here i will pass the name of the embedding model
  sentence_model = SentenceTransformer(model)
  #Embed every sample every row in  dataset output column
  outputs = [example["output"] for example in dataset["train"]]
  # Using the Embedding Model we will convert the text into embeddings
  print("Convert the text to embeddings....")
  embeddings = sentence_model.encode(outputs, show_progress_bar=True)
  dimensions = embeddings.shape[1]
  print("Dimensions of the embedding", embeddings.shape)
  #Create an index using the Faiss as our Vector Database
  index = faiss.IndexFlatIP(dimensions)
  #Normalize the Embeddings
  normalized_embeddings = embeddings / np.linalg.norm(embeddings, axis=1, keepdims=True)
  index.add(normalized_embeddings)
  print("Filtering out near duplicates....")
  #k=2, means atmost two vectors
  D, I = index.search(normalized_embeddings, k=2)
  #In the below list, we will add the list of samples we want to keep
  to_keep=[]
  #We will define the threshold below, if the embedding is 95% similar to other embedding then we will remove that embedding
  for i in tqdm(range(len(embeddings)), desc="Filtering"):
    #If the second closest vector (D[i,1]) has cosine similarity above the threshold
    if D[i,1] >= threshold:
      #Check if the current item or its nearest neighbor is already in the to_keep list
      nearest_neighbor = I[i,1]
      if i not in to_keep and nearest_neighbor not in to_keep:
        # If not, add the current item to the list
        to_keep.append(i)
    else:
        # If the similarity is below the threshold, always keep the current item
        to_keep.append(i)
  print("List", to_keep)
  dataset = dataset["train"].select(to_keep)
  print(dataset.to_pandas())
  return DatasetDict({"train": dataset})

deduped_dataset = deduplicate_dataset(dataset, EMBEDDING_MODEL, 0.95) ## Defined in 03. Set Variables 

In [ ]:
deduped_dataset["train"].to_pandas()
print("Number of rows in the Original Dataset", (len(dataset["train"])))
print("Number of rows in the deduped dataset", (len(deduped_dataset["train"])))
print(f"Number of rows removed: {len(dataset['train']) - len(deduped_dataset['train'])}")

In [ ]:
#Tokenize each row in the Instruction and Output Columns in the  Dataset and Count the Total Number of Tokens
instruction_tokens_count = [len(tokenizer.tokenize(example["instruction"])) for example in deduped_dataset["train"]]
print("Instruction Tokens Count", instruction_tokens_count)
print("Length of Instruction Tokens Count", len(instruction_tokens_count))

output_tokens_count = [len(tokenizer.tokenize(example["output"])) for example in deduped_dataset["train"]]
print("Output Tokens Count", output_tokens_count)
print("Length of the Output Tokens Count", len(output_tokens_count))

#Combine the Instruction Token Count and the Output Token Count
combine_tokens_count = [instruction + output for instruction, output in zip(instruction_tokens_count, output_tokens_count)]
print("Combine Tokens Count", combine_tokens_count)
print("Length of the Combine Tokens Count", len(combine_tokens_count))

#**08. Top K-Sampling**

In Top K-Sampling i will separate the Top 1000 rows from my dataset based on the number of tokens.
So as my daraset has 18,168 rows So, i will take those 1000 rows which have the most number of tokens

In [ ]:
#Get the Top 1000 rows with most number of tokens
def get_top_k_rows(dataset, tokens_count, k):
  #Sort by descending token count and get the top 1000 rows
  sorted_indices = sorted(range(len(tokens_count)), key=lambda i: tokens_count[i], reverse=True)
  top_k_indices = sorted_indices[:k]
  print("Top K Indices", top_k_indices)
  print("Length of Top K Indices", len(top_k_indices))

  #Extract the Top K rows
  top_k_data = {
      "instruction": [dataset["train"][i]["instruction"] for i in top_k_indices],
      "output": [dataset["train"][i]["output"] for i in top_k_indices]
  }
  return Dataset.from_dict(top_k_data)

k=1000
top_k_dataset = get_top_k_rows(deduped_dataset, combine_tokens_count, k)

In [ ]:
dataset = DatasetDict({"train": top_k_dataset})
dataset["train"].to_pandas()

In [ ]:
instruction_tokens_count = [len(tokenizer.tokenize(example["instruction"])) for example in dataset["train"]]
print("Instruction Tokens Count", instruction_tokens_count)
print("Length of Instruction Tokens Count", len(instruction_tokens_count))


output_tokens_count = [len(tokenizer.tokenize(example["output"])) for example in dataset["train"]]
print("Output Tokens Count", output_tokens_count)
print("Length of the Output Tokens Count", len(output_tokens_count))


#Combine the Instruction Token Count and the Output Token Count
combine_tokens_count = [instruction + output for instruction, output in zip(instruction_tokens_count, output_tokens_count)]
print("Combine Tokens Count", combine_tokens_count)
print("Length of the Combine Tokens Count", len(combine_tokens_count))

In [ ]:
#Combine Tokens Count
plot_distribution(combine_tokens_count, "Distribution of the tokens count for combined Instruction + Output Column")


In [ ]:
# Read as pandas DataFrame
dataset['train'].to_pandas()

# TODO

- Add more code to Get Info
- Change this to a CLI from the notebook

# Resources and Credit

## Original Notebook from Muhammad Moin
> Link to original notebook and video 

- https://github.com/MuhammadMoinFaisal/LargeLanguageModelsProjects/blob/main/Fine-Tune-Llama2/Dataset_Creating_For_Fine_Tuning_Llama_2.ipynb
- [Efficient Fine-Tuning for Llama 2 on Custom Dataset with QLoRA on a Single GPU in Google Colab](https://www.youtube.com/watch?v=YyZqcNo4hdo&t=122s)

## Link to Huggingface Model and Dataset in Notebook

- https://huggingface.co/MMoin
- https://huggingface.co/datasets/MMoin/mini-platypus-two
